# Mean Field Game – Pipeline Calibrado

Este notebook:

1. Carrega os dados limpos da B3.
2. Recalibra nu/phi/gamma_T com heurísticas mais conservadoras.
3. Configura o grid/solver e executa o MFG (com opção de preço endógeno).
4. Salva artefatos e gráficos em `notebooks_output/`.

In [1]:
from __future__ import annotations

import json
from datetime import datetime
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yaml

from mfg_finance.grid import Grid1D
from mfg_finance.models.hft import HFTParams, eta_from_m_alpha, initial_density
from mfg_finance.price import solve_price_clearing
from mfg_finance.solver import solve_mfg_picard
from mfg_finance.viz import plot_alpha_cuts, plot_convergence, plot_density_time, plot_price, plot_value_time

In [4]:
ROOT = Path.cwd().parent
DATA_PROCESSED = ROOT / 'data' / 'processed'
OUTPUT_BASE = ROOT / 'notebooks_output'
OUTPUT_BASE.mkdir(parents=True, exist_ok=True)

timestamp = datetime.now().strftime('run-%Y%m%d-%H%M%S')
REPORT_DIR = OUTPUT_BASE / timestamp
REPORT_DIR.mkdir(parents=True, exist_ok=True)
REPORT_DIR

WindowsPath('c:/Users/Cockles/Documents/GitHub/mfg-for-financial-market/notebooks_output/run-20251019-194056')

In [5]:
# Calibração heurística (valores mais brandos para estabilidade)
asset_summary = pd.read_csv(DATA_PROCESSED / 'cotahist_asset_summary.csv')
asset_summary = asset_summary.replace([np.inf, -np.inf], np.nan).dropna(subset=['std_log_return', 'annualised_vol'])
median_daily_std = float(asset_summary['std_log_return'].median())
median_annual_vol = float(asset_summary['annualised_vol'].median())
upper_quartile_vol = float(asset_summary['annualised_vol'].quantile(0.75))

nu = median_daily_std ** 2
phi = median_annual_vol / 20.0
gamma_T = upper_quartile_vol * 0.5

calibration_summary = {
    'median_daily_std': median_daily_std,
    'median_annual_vol': median_annual_vol,
    'upper_quartile_vol': upper_quartile_vol,
    'nu': round(nu, 6),
    'phi': round(phi, 6),
    'gamma_T': round(gamma_T, 6),
}
with open(REPORT_DIR / 'calibration.json', 'w', encoding='utf-8') as fp:
    json.dump(calibration_summary, fp, indent=2)
calibration_summary

{'median_daily_std': 0.0445486261341157,
 'median_annual_vol': 0.7071875160027775,
 'upper_quartile_vol': 1.137724647706284,
 'nu': 0.001985,
 'phi': 0.035359,
 'gamma_T': 0.568862}

In [6]:
# Atualiza baseline.yaml (opcional)
baseline_path = ROOT / 'mfg-finance' / 'configs' / 'baseline.yaml'
with open(baseline_path, 'r', encoding='utf-8') as fp:
    baseline_cfg = yaml.safe_load(fp)
baseline_cfg.setdefault('params', {})
baseline_cfg['params']['nu'] = calibration_summary['nu']
baseline_cfg['params']['phi'] = calibration_summary['phi']
baseline_cfg['params']['gamma_T'] = calibration_summary['gamma_T']
with open(baseline_path, 'w', encoding='utf-8') as fp:
    yaml.safe_dump(baseline_cfg, fp, sort_keys=False)
baseline_cfg['params']

{'nu': 0.001985,
 'phi': 0.035359,
 'gamma_T': 0.568862,
 'eta0': 0.5,
 'eta1': 0.8,
 'm0_mean': 0.0,
 'm0_std': 1.0}

In [8]:
# Construção do grid/parâmetros (baseline leve)
grid_cfg = baseline_cfg['grid']
solver_cfg = baseline_cfg.get('solver', {})
params_cfg = baseline_cfg['params']
grid = Grid1D(
    x_min=float(grid_cfg['x_min']),
    x_max=float(grid_cfg['x_max']),
    nx=int(grid_cfg['nx']),
    T=float(grid_cfg['T']),
    nt=int(grid_cfg['nt']),
    bc=str(grid_cfg.get('bc', 'neumann')),
)
params = HFTParams(
    nu=float(params_cfg['nu']),
    phi=float(params_cfg['phi']),
    gamma_T=float(params_cfg['gamma_T']),
    eta0=float(params_cfg.get('eta0', 0.5)),
    eta1=float(params_cfg.get('eta1', 0.8)),
    m0_mean=float(params_cfg.get('m0_mean', 0.0)),
    m0_std=float(params_cfg.get('m0_std', 1.0)),
)
grid, params

(Grid1D(x_min=-5.0, x_max=5.0, nx=201, T=0.5, nt=150, bc='neumann', dx=0.04999999999999982, dt=0.0033333333333333335, x=array([-5.  , -4.95, -4.9 , -4.85, -4.8 , -4.75, -4.7 , -4.65, -4.6 ,
        -4.55, -4.5 , -4.45, -4.4 , -4.35, -4.3 , -4.25, -4.2 , -4.15,
        -4.1 , -4.05, -4.  , -3.95, -3.9 , -3.85, -3.8 , -3.75, -3.7 ,
        -3.65, -3.6 , -3.55, -3.5 , -3.45, -3.4 , -3.35, -3.3 , -3.25,
        -3.2 , -3.15, -3.1 , -3.05, -3.  , -2.95, -2.9 , -2.85, -2.8 ,
        -2.75, -2.7 , -2.65, -2.6 , -2.55, -2.5 , -2.45, -2.4 , -2.35,
        -2.3 , -2.25, -2.2 , -2.15, -2.1 , -2.05, -2.  , -1.95, -1.9 ,
        -1.85, -1.8 , -1.75, -1.7 , -1.65, -1.6 , -1.55, -1.5 , -1.45,
        -1.4 , -1.35, -1.3 , -1.25, -1.2 , -1.15, -1.1 , -1.05, -1.  ,
        -0.95, -0.9 , -0.85, -0.8 , -0.75, -0.7 , -0.65, -0.6 , -0.55,
        -0.5 , -0.45, -0.4 , -0.35, -0.3 , -0.25, -0.2 , -0.15, -0.1 ,
        -0.05,  0.  ,  0.05,  0.1 ,  0.15,  0.2 ,  0.25,  0.3 ,  0.35,
         0.4 ,  0.45,  0.5 , 

In [9]:
# Executa o solver com salvaguardas numéricas
m0 = initial_density(grid, params)
hjb_kwargs = {
    'max_inner': 3,
    'tol': 1e-8,
    'max_dissipation': 1.0,
    'alpha_cap': 1.0,
    'value_cap': 40.0,
    'value_relaxation': 0.5,
}
solver_mix = float(solver_cfg.get('mix', 0.6))
U_all, M_all, alpha_all, errors, metrics = solve_mfg_picard(
    grid,
    params,
    max_iter=int(solver_cfg.get('max_iter', 120)),
    tol=float(solver_cfg.get('tol', 1e-6)),
    mix=solver_mix,
    m0=m0,
    hjb_kwargs=hjb_kwargs,
    eta_callback=eta_from_m_alpha,
)
run_metrics = {
    'iterations': len(errors),
    'final_error': float(errors[-1]) if errors else None,
    **metrics,
}
run_metrics

{'iterations': 120,
 'final_error': 113.1437996319017,
 'mean_abs_alpha': 0.8181875258917322,
 'std_alpha': 3.3190957227011433,
 'liquidity_proxy': 0.6068200556999621}

In [10]:
# Clearing de preço (opcional)
compute_price = True
price_results = None
if compute_price:
    supply_schedule = np.zeros(len(grid.t))
    sensitivity = 0.2
    def alpha_field(idx: int, price: float) -> np.ndarray:
        return alpha_all[idx] - sensitivity * price
    prices = solve_price_clearing(alpha_field, M_all, supply_schedule, grid.dx)
    price_results = prices
    run_metrics['price_mean'] = float(np.mean(prices))
    run_metrics['price_std'] = float(np.std(prices))
run_metrics

{'iterations': 120,
 'final_error': 113.1437996319017,
 'mean_abs_alpha': 0.8181875258917322,
 'std_alpha': 3.3190957227011433,
 'liquidity_proxy': 0.6068200556999621,
 'price_mean': -5.502418646749282,
 'price_std': 0.3636086248062095}

In [11]:
# Persistência de artefatos
np.save(REPORT_DIR / 'U_all.npy', U_all)
np.save(REPORT_DIR / 'M_all.npy', M_all)
np.save(REPORT_DIR / 'alpha_all.npy', alpha_all)
with open(REPORT_DIR / 'metrics.json', 'w', encoding='utf-8') as fp:
    json.dump(run_metrics, fp, indent=2)
if price_results is not None:
    np.savetxt(REPORT_DIR / 'price.csv', np.column_stack((grid.t, price_results)), delimiter=',', header='time,price', comments='')
sorted(p.name for p in REPORT_DIR.iterdir())

['M_all.npy',
 'U_all.npy',
 'alpha_all.npy',
 'calibration.json',
 'metrics.json',
 'price.csv']

In [12]:
# Gráficos padrão
plot_density_time(M_all, grid, REPORT_DIR / 'density.png')
plot_value_time(U_all, grid, REPORT_DIR / 'value.png')
plot_alpha_cuts(alpha_all, grid, times=[0.0, 0.25 * grid.T, 0.5 * grid.T], path=REPORT_DIR / 'alpha_cuts.png')
plot_convergence(errors, REPORT_DIR / 'convergence.png')
if price_results is not None:
    plot_price(grid.t, price_results, REPORT_DIR / 'price.png')
sorted(p.name for p in REPORT_DIR.iterdir())

['M_all.npy',
 'U_all.npy',
 'alpha_all.npy',
 'alpha_cuts.png',
 'calibration.json',
 'convergence.png',
 'density.png',
 'metrics.json',
 'price.csv',
 'price.png',
 'value.png']

In [13]:
# Resumo final amigável
summary_display = {
    'output_dir': str(REPORT_DIR),
    'iterations': run_metrics.get('iterations'),
    'final_error': run_metrics.get('final_error'),
    'mean_abs_alpha': run_metrics.get('mean_abs_alpha'),
    'std_alpha': run_metrics.get('std_alpha'),
    'liquidity_proxy': run_metrics.get('liquidity_proxy'),
    'price_mean': run_metrics.get('price_mean'),
    'price_std': run_metrics.get('price_std'),
}
summary_display

{'output_dir': 'c:\\Users\\Cockles\\Documents\\GitHub\\mfg-for-financial-market\\notebooks_output\\run-20251019-194056',
 'iterations': 120,
 'final_error': 113.1437996319017,
 'mean_abs_alpha': 0.8181875258917322,
 'std_alpha': 3.3190957227011433,
 'liquidity_proxy': 0.6068200556999621,
 'price_mean': -5.502418646749282,
 'price_std': 0.3636086248062095}